In [1]:
import pandas as pd
import sqlalchemy as sal
engine=sal.create_engine(r"mssql://KALIVARUN\MSSQL/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER")
conn=engine.connect()

In [38]:
def extract():
   df_products=pd.read_csv("products.txt")
   df_product_dim = pd.read_sql_query("select * from product_dim" ,conn)
   return(df_products,df_product_dim)


def transform():
    df_merge=pd.merge(df_products,df_product_dim,how="inner",on="product_id")
    # df_product_dim["startdate"]='2025-05-01'
    # df_product_dim["enddate"]='9999-12-31'
    # df_insert=
    return (df_merge)

In [39]:
df_products,df_product_dim=extract()

In [24]:
df_product_dim

,product_key,product_id,product_name,price,startdate,enddate


In [28]:
df_products

,product_id,product_name,price
0,1,Iphone13,55000
1,2,Samsung,40000


In [40]:
df_merge=transform()

In [31]:
df_merge

,product_id,product_name_x,price_x,product_key,product_name_y,price_y,startdate,enddate


In [41]:

#first time
df_products["startdate"]='2025-05-01'
df_products["enddate"]='9999-12-31'

In [42]:
df_products

,product_id,product_name,price,startdate,enddate
0,100,Iphone13,55000,2025-05-01,9999-12-31
1,200,Samsung,40000,2025-05-01,9999-12-31


In [43]:
#insert into dim
df_products.to_sql("product_dim",con=conn,index=False,if_exists="append")
conn.commit()

In [52]:
def extract():
   df_products=pd.read_csv("products.txt")
   df_product_dim = pd.read_sql_query("select * from product_dim where enddate='9999-12-31'" ,conn)
   return (df_products,df_product_dim)

def transform(df_products,df_product_dim):
    df_merged = pd.merge(df_products , df_product_dim , how='inner' , on = 'product_id')
    update_rows= df_merged['product_key']
    keys = update_rows.to_list()
    product_keys= ','.join([str(key) for key in keys])
    return product_keys

def updates(product_keys):
    query = sal.text("update product_dim set enddate =  cast(getdate()-1 as date) where product_key in (" + product_keys + ")")
    p = conn.execute(query)
    conn.commit()    
    
def inserts(df_products):
    df_products["startdate"]=pd.to_datetime('now').strftime('%Y-%m-%d')
    df_products["enddate"]='9999-12-31'
    df_insert= df_products.to_sql("product_dim",con=conn,index=False,if_exists="append")
    conn.commit()
    return (df_insert)

In [53]:
df_products,df_product_dim = extract()

#%%
product_keys = transform(df_products,df_product_dim)

#product_keys= transform(df_products,df_product_dim)
#%%
if product_keys != '':
    updates(product_keys)
#%%
inserts(df_products)


2